In [1]:
import os
import pprint
import tempfile
import datetime
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [2]:
train_filename = "../data/samples/train_transactions.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "../data/samples/test_transactions.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

validation_filename = "../data/samples/validation_transactions.tfrecord"
validation = tf.data.TFRecordDataset(validation_filename)

feature_description = {
    'context_item_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),    
    'context_item_quantity': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_item_price': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),    
    'context_item_department_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'label_item_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)
def _map_function(x):
  return {
    "context_item_id": tf.strings.as_string(x["context_item_id"]),
    "context_item_price": x["context_item_price"],
    "context_item_department_id": x["context_item_department_id"],
    "label_item_id": tf.strings.as_string(x["label_item_id"])}
    
train_ds = train.map(_parse_function).map(_map_function)
test_ds = test.map(_parse_function).map(_map_function)
cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()
cached_validation = validation.map(_parse_function).map(_map_function).batch(1280).cache()

In [3]:
items_filename = "../data/samples/items.tfrecord"
items_tf = tf.data.TFRecordDataset(items_filename)
item_feature_description = {
    'item_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'item_fullprice': tf.io.FixedLenFeature([1], tf.float32, default_value=0.0 )
}

def item_parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, item_feature_description)

items_ds = items_tf.map(item_parse_function).map(lambda x: {
    "item_id": tf.strings.as_string(x["item_id"]),
    "item_fullprice": x["item_fullprice"]
})
item_ids = items_ds.map(lambda x: x["item_id"]).batch(1_000)
unique_item_ids = np.unique(np.concatenate(list(item_ids)))
item_prices = np.concatenate(list(items_ds.map(lambda x: x["item_fullprice"]).batch(1000)))
max_price = item_prices.max()
min_price = item_prices.min()

price_buckets = np.linspace(
    min_price, max_price, num=40,
)

dept_filename = "../data/samples/departments.tfrecord"
departments_tf = tf.data.TFRecordDataset(dept_filename)
dept_feature_description = {
  'dept_id' : tf.io.FixedLenFeature([1], tf.int64, default_value=0)
}
def dept_parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, dept_feature_description)
department_ids = np.concatenate(list(departments_tf.map(dept_parse_function).map(lambda x: x['dept_id'])))

In [4]:
class ItemEmbeddingModel(tf.keras.Model):
  embedding_dimension = 32
  def __init__(self):
    super().__init__()

    self.item_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(vocabulary=unique_item_ids, mask_token=None, name='item_id_string_lookup'),
      tf.keras.layers.Embedding(len(unique_item_ids) + 1, self.embedding_dimension, name='item_id_embedding'),
      tf.keras.layers.GRU(self.embedding_dimension, name='item_id_rnn'),
    ], name='item_embedding')

    self.price_normalization = tf.keras.layers.Discretization(price_buckets.tolist())

    self.price_embedding = tf.keras.Sequential([
      self.price_normalization,
      tf.keras.layers.Embedding(len(price_buckets)+1,output_dim=self.embedding_dimension),
      tf.keras.layers.GRU(self.embedding_dimension, name='item_price_rnn')
    ], name='price_embedding')    

    self.dept_categorical = tf.keras.layers.IntegerLookup(max_tokens=len(department_ids)+1, vocabulary=department_ids)

    self.dept_embedding = tf.keras.Sequential([
      self.dept_categorical,
      tf.keras.layers.Embedding(len(department_ids)+1, output_dim=self.embedding_dimension),
      tf.keras.layers.GRU(self.embedding_dimension, name='dept_rnn')
    ])

  def call(self, features):
    return tf.concat([
        self.item_embedding(features["context_item_id"]),
        self.price_embedding(features["context_item_price"]),
        self.dept_embedding(features['context_item_department_id'])
    ], axis=1)

class DeepLayerModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes, embedding_model):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = embedding_model

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [5]:
class RetrievalModel(tfrs.Model):
    def __init__(self, deep_layers):
        self.embedding_dim = 32
        self.output_dim = 32
        super().__init__()
        self._query_model = DeepLayerModel(deep_layers, ItemEmbeddingModel())
        self._candidate_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_item_ids, mask_token=None, name='candidate_itemid_lookup'),
            tf.keras.layers.Embedding(len(unique_item_ids) + 1, self.embedding_dim, name='candidate_embedding_lookup'),
            tf.keras.layers.Dense(self.output_dim)
            ], name='candidate_model')
        metrics = tfrs.metrics.FactorizedTopK(candidates=items_tf.batch(128).map(self._candidate_model))
        self._task = tfrs.tasks.Retrieval(metrics=metrics)

    def compute_loss(self, features, training=False):
        item_history = {
            "context_item_id": features["context_item_id"],
            "context_item_price": features["context_item_price"],
            "context_item_department_id" : features["context_item_department_id"]}   
        next_item_label = features["label_item_id"]

        query_embedding = self._query_model(item_history)       
        candidate_embedding = self._candidate_model(next_item_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

In [7]:
log_dir = '../logs/fit/final_model'
deep_layers = [32]
epochs = 60
learning_rate = 0.09
model = RetrievalModel(deep_layers=deep_layers)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=learning_rate))
# model.fit(cached_train, epochs=epochs, validation_data=cached_validation, callbacks=[tensorboard_callback])
model.fit(cached_train,epochs=epochs)
evaluation = model.evaluate(cached_test, return_dict=True)
print(evaluation)

Epoch 1/60
78/78 [==============================] - 15s 134ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 113510.6860 - regularization_loss: 0.0000e+00 - total_loss: 113510.6860
Epoch 2/60
78/78 [==============================] - 10s 134ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 102466.1547 - regularization_loss: 0.0000e+00 - total_loss: 102466.1547
Epoch 3/60
78/78 [==============================] - 10s 133ms/step - factorized_top_k/top_1_categorical_accuracy: 0.00

In [8]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model._query_model)
# recommends movies out of the entire movies dataset.
items = items_ds.map(lambda x: x["item_id"])
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model._candidate_model)))
)

feature0 = ["76538","42573","130819","141002","102841","119889","10139","28703","104876","114395"]
feature1 = [10.59,1.49,6.99,6.99,135.99,159.99,11.99,17.99,5.29,1.79]
feature2 = [673,671,683,748,687,687,673,695,658,752]
features_dataset = { 
  'context_item_id' : tf.constant(feature0,shape=(1,10,1)), 
  'context_item_price' : tf.constant(feature1,shape=(1,10,1)),
  'context_item_department_id' : tf.constant(feature2, shape=(1,10,1))
}

_, predicted = index(features_dataset)
print(predicted)

tf.Tensor(
[[[b'98816']
  [b'114395']
  [b'13888']
  [b'114411']
  [b'92788']
  [b'107835']
  [b'13911']
  [b'23167']
  [b'92790']
  [b'13939']]], shape=(1, 10, 1), dtype=string)


In [9]:
path = '../data/rich_2features_model'
tf.saved_model.save(index, path)

INFO:tensorflow:Assets written to: ../data/rich_2features_model\assets


INFO:tensorflow:Assets written to: ../data/rich_2features_model\assets
